Необходимо собрать информацию по продуктам питания с сайта: Список протестированных продуктов на сайте Росконтроль.рф Приложение должно анализировать несколько страниц сайта (вводим через input или аргументы).
Получившийся список должен содержать:

- Наименование продукта.
- Все параметры (Безопасность, Натуральность, Пищевая ценность, Качество) Не забываем преобразовать к цифрам
- Общую оценку
- Сайт, откуда получена информация.


Общий результат можно вывести с помощью dataFrame через Pandas. Сохраните в json либо csv.

In [54]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup as bs

In [55]:
url = 'https://roscontrol.com'
route = '/testlab/search'
params = {
  'keyword': 'Хлеб',
  'page': 1
}
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'}

In [56]:
products = []

while True:
  response = requests.get(url + route, params=params, headers=headers)
  if response.ok:
    dom = bs(response.text, 'html.parser')

    page_products = dom.select('div.wrap-product-catalog__item')

    for page_product in page_products:
      product_data = {
        'title': None,
        'rating': {},
      }

      rating_total_score_node = page_product.find('div', {'class': ['rate']})

      product_data['source_url'] = url
      product_data['title'] = page_product.find('div', {'class': 'product__item-link'}).text

      if rating_total_score_node:
        if rating_total_score_node.text:
          product_data['rating']['total'] = rating_total_score_node.text
          rating_scores = page_product.find('div', {'class': 'rating-block'}).findChildren('div', recursive=False)
          for rating_score in rating_scores:
            rating_label = rating_score.find('div', {'class': 'left'}).find('div', {'class': 'text'}).text
            product_data['rating'][rating_label] = rating_score.find('div', {'class': 'right'}).text
        else:
          product_data['rating']['total'] = -1
          blacklist_node = page_product.find('div', {'class': 'rating-block'}).find('div', {'class': 'blacklist__item-danger'})
          product_data['rating']['Степень_нарушения'] = blacklist_node.find_all('span', {'class': ['def-text']})[1].text

      products.append(product_data)

    print(f"Обработано продуктов на {params['page']} странице: {len(page_products)}")
    params['page'] += 1
  else:
    break


Обработано продуктов на 1 странице: 12
Обработано продуктов на 2 странице: 12
Обработано продуктов на 3 странице: 12
Обработано продуктов на 4 странице: 12
Обработано продуктов на 5 странице: 12
Обработано продуктов на 6 странице: 12
Обработано продуктов на 7 странице: 12
Обработано продуктов на 8 странице: 12
Обработано продуктов на 9 странице: 12
Обработано продуктов на 10 странице: 12
Обработано продуктов на 11 странице: 12
Обработано продуктов на 12 странице: 12
Обработано продуктов на 13 странице: 12
Обработано продуктов на 14 странице: 12
Обработано продуктов на 15 странице: 12
Обработано продуктов на 16 странице: 12
Обработано продуктов на 17 странице: 12
Обработано продуктов на 18 странице: 12
Обработано продуктов на 19 странице: 12
Обработано продуктов на 20 странице: 12
Обработано продуктов на 21 странице: 12
Обработано продуктов на 22 странице: 12
Обработано продуктов на 23 странице: 12
Обработано продуктов на 24 странице: 12
Обработано продуктов на 25 странице: 12
Обработан

In [64]:
data = pd.DataFrame(products)
data.sample(10)

,title,rating,source_url
501,"Хлеб ""Черемушки"" ""Бородинский""","{'total': ' 72 ', 'Безопасность': '83', 'Натур...",https://roscontrol.com
982,"Хлеб ""То, что надо!"" ""Дарницкий"" в нарезке","{'total': ' 71 ', 'Безопасность': '80', 'Натур...",https://roscontrol.com
812,"Хлеб ""Черемушки"" ""Дарницкий"" в нарезке","{'total': ' 68 ', 'Безопасность': '90', 'Натур...",https://roscontrol.com
449,"Квас ""Останкинский"" ""Хлебный""","{'total': ' 77 ', 'Безопасность': '66', 'Натур...",https://roscontrol.com
25,Сушки Fine Life с маком,"{'total': ' 57 ', 'Безопасность': '40', 'Натур...",https://roscontrol.com
195,Хлебопечка Stadler Form Baker Two SFBM.9900,{},https://roscontrol.com
96,"Хлеб ""Пролетарец"" ""Дарницкий""","{'total': -1, 'Степень_нарушения': 'Средняя'}",https://roscontrol.com
507,"Хлеб ""Маркет"" ""Бородинский""","{'total': ' 70 ', 'Безопасность': '85', 'Натур...",https://roscontrol.com
724,"Хлеб Billa ""Бородинский""","{'total': ' 68 ', 'Безопасность': '83', 'Натур...",https://roscontrol.com
962,"Хлеб ""Коломенское"", ""Бородинский""","{'total': ' 66 ', 'Безопасность': '87', 'Натур...",https://roscontrol.com


In [58]:
with open('products.json', 'w') as f:
    json.dump(products, f)